**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [ ]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten
import skvideo
import skvideo.io
import collections

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [ ]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [83]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_state, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

act is basically being used to select the next action. In order to choose the next action to be used, we use an epsilon greedy algorithm to select the same. Depending on a random number generated, we choose to either use the knowledge of best action to take that maximes the cumulative reward if its lower than epsilon, or if it's greater we explore new actions. 

This helps achieve a tradeoff between exploring new actions and using the best knowledge, both of which are important

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [7]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [27]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=15 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.



Position
This matrix is used to define the rat position and allowed moves and it is equal to 1 where the rat actually is and 0 everywhere else. Also uses -1 which form the frame

Board
This matrix is used to define the environment. This matrix contains the rewards for the cat with cheese and poison.



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [5]:
class RandomAgent(Agent):
    def __init__(self, n_action=4):
        super(RandomAgent, self).__init__(n_action)
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]  #Chooses a random action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [3]:
def test(agent, env, epochs, prefix='', frame=5):
    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
                
        ##### FILL IN HERE
        
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        #Play game until it is over which is given by the env
        while not game_over:
            action = agent.learned_act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward
                
        # Save as a mp4
        env.draw(prefix+str(e))
                
        # Update stats
        score += win-lose
                
            
        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))

    print('Final score: '+str(score/epochs))

In [28]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/12.0. Average score (-0.5)
Win/lose count 11.0/16.0. Average score (-2.0)
Win/lose count 12.5/17.0. Average score (-2.625)
Win/lose count 10.0/16.0. Average score (-3.3)
Win/lose count 12.0/16.0. Average score (-3.4166666666666665)
Win/lose count 8.5/12.0. Average score (-3.4285714285714284)
Win/lose count 18.0/13.0. Average score (-2.375)
Win/lose count 10.0/11.0. Average score (-2.2222222222222223)
Win/lose count 7.5/18.0. Average score (-3.05)
Win/lose count 8.5/16.0. Average score (-3.4545454545454546)
Final score: -3.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__1)__
$$
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + E_{(s',a')\sim p(.|s,a)}\left(\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}
$$


We have $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$, which is deterministic, therefore :
$$
\begin{align*}
Q^*(s,a)&= E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a)\,and\,a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}
$$ 

We've shown that $Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$, therefore, $Q^*$ is the solution of the equation $F(Q) = 0$ with $$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

Minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will hence lead us to find the optimal Q.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [15]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = collections.deque(maxlen=max_memory) 
        
    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [14]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Update stats
        score += win-lose
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
            
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [103]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    # add epsilon decay method for last question
    def set_epsilon(self, e):
        self.epsilon *= e # e is the decay percentage
        
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # sample minibatch of transitions
            # (state, next_state, action, reward, game_over)
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_ #Update final reward
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, 0))) #Update reward with discount factor
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        adam = Adam(lr=lr)
        model.compile(adam, "mse")
        self.model = model

In [104]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(grid_size=size, lr=.01, epsilon = 0.1, memory_size=50000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/015 | Loss 0.0040 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/015 | Loss 0.0068 | Win/lose count 11.5/3.0 (8.5)
Epoch 002/015 | Loss 0.0034 | Win/lose count 11.0/8.0 (3.0)
Epoch 003/015 | Loss 0.0071 | Win/lose count 13.5/6.0 (7.5)
Epoch 004/015 | Loss 0.0069 | Win/lose count 13.0/6.0 (7.0)
Epoch 005/015 | Loss 0.0050 | Win/lose count 6.5/5.0 (1.5)
Epoch 006/015 | Loss 0.0037 | Win/lose count 16.5/3.0 (13.5)
Epoch 007/015 | Loss 0.1041 | Win/lose count 9.0/4.0 (5.0)
Epoch 008/015 | Loss 0.0023 | Win/lose count 13.5/1.0 (12.5)
Epoch 009/015 | Loss 0.0019 | Win/lose count 16.0/1.0 (15.0)
Epoch 010/015 | Loss 0.0037 | Win/lose count 13.5/5.0 (8.5)
Epoch 011/015 | Loss 0.0023 | Win/lose count 11.0/3.0 (8.0)
Epoch 012/015 | Loss 0.0028 | Win/lose count 7.0/2.0 (5.0)
Epoch 013/015 | Loss 0.0056 | Win/lose count 8.5/5.0 (3.5)
Epoch 014/015 | Loss 0.0027 | Win/lose count 9.5/1.0 (8.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [101]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__( *args,**kwargs)
        
        model = Sequential()
        #Best model
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [102]:
epochs_train = 15
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=5000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/015 | Loss 0.0013 | Win/lose count 1.5/0 (1.5)
Epoch 001/015 | Loss 0.0007 | Win/lose count 5.0/7.0 (-2.0)
Epoch 002/015 | Loss 0.0019 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/015 | Loss 0.0059 | Win/lose count 2.0/2.0 (0.0)
Epoch 004/015 | Loss 0.0022 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/015 | Loss 0.0084 | Win/lose count 3.5/5.0 (-1.5)
Epoch 006/015 | Loss 0.0050 | Win/lose count 2.5/6.0 (-3.5)
Epoch 007/015 | Loss 0.0182 | Win/lose count 3.0/6.0 (-3.0)
Epoch 008/015 | Loss 0.0071 | Win/lose count 8.5/0 (8.5)
Epoch 009/015 | Loss 0.0019 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/015 | Loss 0.0071 | Win/lose count 5.0/4.0 (1.0)
Epoch 011/015 | Loss 0.0008 | Win/lose count 2.5/1.0 (1.5)
Epoch 012/015 | Loss 0.0033 | Win/lose count 5.5/5.0 (0.5)
Epoch 013/015 | Loss 0.1129 | Win/lose count 9.5/1.0 (8.5)
Epoch 014/015 | Loss 0.0016 | Win/lose count 6.0/3.0 (3.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [38]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0.5/1.0. Average score (-0.25)
Win/lose count 2.5/0. Average score (0.6666666666666666)
Win/lose count 1.5/0. Average score (0.875)
Win/lose count 1.0/1.0. Average score (0.7)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 1.0/0. Average score (0.7857142857142857)
Win/lose count 0/0. Average score (0.6875)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 0.5/0. Average score (0.65)
Win/lose count 0.5/0. Average score (0.6363636363636364)
Final score: 0.7
Test of the FC
Win/lose count 1.0/4.0. Average score (-1.5)
Win/lose count 2.0/5.0. Average score (-2.0)
Win/lose count 1.5/2.0. Average score (-1.625)
Win/lose count 0.5/3.0. Average score (-1.8)
Win/lose count 0.5/1.0. Average score (-1.5833333333333333)
Win/lose count 0/1.0. Average score (-1.5)
Win/lose count 2.5/5.0. Average score (-1.625)
Win/lose count 1.0/2.0. Average score (-1.5555555555555556)
Win/lose count 0/3.0. Average score (-1.7)
Win/lose count 1.5/2.0. Av

In [39]:
HTML(display_videos('cnn_test10.mp4'))

In [40]:
HTML(display_videos('fc_test10.mp4'))

CNN outperform FC , however they both end up stuck in a position, keeps oscillating between two positions and thus we need to force the agent to explore in plus. 

Temperature increases the density of the reward pixel. Therefore it just increases the score. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [33]:
def train_explore(agent, env, epoch, epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        win, lose = 0, 0
        agent.set_epsilon(epsilon)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(decay) # we use epsilon decay to make epsilon decrease over time
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
        # Update stats
        score += win-lose
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:3.2f}/{:3.2f} ({:3.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the rat
        self.x = 0
        self.y = 1
        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0]    = 256
        b[self.board<0, 2] = 256
        b[self.x,self.y,:]   = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:]  = 0
        b[:,:2,:]  = 0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 
        self.position[:, -2:] = -1
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        self.board[self.x, self.y] = 0 # erase reward from board
        game_over = self.t > self.max_time # game over if more than 500 steps

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # add 0.5 of reward in some cases of the board
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        # add -1 reward in some cases of the board
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 # if both malus and bonus on the same case -> put 0

        self.board = bonus + malus # board is 0 if no reward, -0.5 if malus, 1 if positive reward
        
        # lines added (put 0 poison/cheese outside the border)
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), #As suggested
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
      
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
 #                               self.board.reshape(self.grid_size, self.grid_size,1),
  #                      self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [76]:
# Training
epochs_train = 30
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3) 
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 0.0195 | Win/lose count 14.00/46.60 (-32.60)
Epoch 001/030 | Loss 0.0159 | Win/lose count 15.50/31.20 (-15.70)
Epoch 002/030 | Loss 0.0236 | Win/lose count 17.00/23.30 (-6.30)
Epoch 003/030 | Loss 0.0099 | Win/lose count 22.00/23.50 (-1.50)
Epoch 004/030 | Loss 0.0140 | Win/lose count 25.50/11.90 (13.60)
Epoch 005/030 | Loss 0.0133 | Win/lose count 21.50/11.30 (10.20)
Epoch 006/030 | Loss 0.0162 | Win/lose count 26.00/11.90 (14.10)
Epoch 007/030 | Loss 0.0454 | Win/lose count 25.00/12.20 (12.80)
Epoch 008/030 | Loss 0.0072 | Win/lose count 28.00/10.90 (17.10)
Epoch 009/030 | Loss 0.0129 | Win/lose count 15.00/12.60 (2.40)
Epoch 010/030 | Loss 0.0205 | Win/lose count 25.50/11.30 (14.20)
Epoch 011/030 | Loss 0.0125 | Win/lose count 23.50/12.70 (10.80)
Epoch 012/030 | Loss 0.0078 | Win/lose count 25.00/10.80 (14.20)
Epoch 013/030 | Loss 0.0160 | Win/lose count 24.00/10.30 (13.70)
Epoch 014/030 | Loss 0.0190 | Win/lose count 24.00/11.50 (12.50)
Epoch 015/030 | Loss 0.0

In [77]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 24.5/3.0. Average score (10.75)
Win/lose count 24.0/11.0. Average score (11.5)
Win/lose count 25.5/10.0. Average score (12.5)
Win/lose count 21.5/8.0. Average score (12.7)
Win/lose count 25.5/8.0. Average score (13.5)
Win/lose count 27.0/7.0. Average score (14.428571428571429)
Win/lose count 20.5/6.0. Average score (14.4375)
Win/lose count 22.5/6.0. Average score (14.666666666666666)
Win/lose count 23.5/9.0. Average score (14.65)
Win/lose count 24.0/10.0. Average score (14.590909090909092)
Final score: 16.05


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [54]:
class Mimic_Agent(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic_Agent, self).__init__(epsilon = epsilon)
        self.discount = 0.3
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_  = self.memory.random_access()
            
            target_q[i, a_] = 5
            input_states[i] = s_
        
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        
class CNN_Mimic_Agent(Mimic_Agent):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(CNN_Mimic_Agent, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
def train_mimic_agent(ref, agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, reset the game
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # Perform an action from the reference agent
            action = ref.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward and check if game over
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [70]:
epochs_train = 50
pro_agent = agent
mimic_agent = CNN_Mimic_Agent(size, lr=.01, epsilon = 0.15, memory_size=5000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimic_agent(pro_agent,mimic_agent, env, epochs_train, prefix='train_cnn_mimick_agent')

Epoch 000/050 | Loss 3.7401 | Win/lose count 24.0/19.69999999999999 (4.300000000000011)
Epoch 001/050 | Loss 3.3806 | Win/lose count 23.0/21.000000000000004 (1.9999999999999964)
Epoch 002/050 | Loss 3.0496 | Win/lose count 17.0/16.99999999999999 (1.0658141036401503e-14)
Epoch 003/050 | Loss 3.8314 | Win/lose count 23.0/15.599999999999984 (7.400000000000016)
Epoch 004/050 | Loss 3.8564 | Win/lose count 21.0/21.50000000000001 (-0.5000000000000107)
Epoch 005/050 | Loss 3.2420 | Win/lose count 20.0/15.799999999999978 (4.200000000000022)
Epoch 006/050 | Loss 3.6913 | Win/lose count 23.0/19.899999999999984 (3.1000000000000156)
Epoch 007/050 | Loss 4.1680 | Win/lose count 26.5/16.99999999999998 (9.500000000000021)
Epoch 008/050 | Loss 2.9198 | Win/lose count 21.5/17.499999999999986 (4.000000000000014)
Epoch 009/050 | Loss 3.4912 | Win/lose count 26.5/17.799999999999994 (8.700000000000006)
Epoch 010/050 | Loss 3.1918 | Win/lose count 22.0/19.29999999999999 (2.70000000000001)
Epoch 011/050 | Lo

In [71]:
# Evaluation
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(mimic_agent,env_test,epochs_test,prefix='test_cnn_mimick_agent')
HTML(display_videos('test_cnn_mimick_agent5.mp4'))

Win/lose count 14.5/4.0. Average score (5.25)
Win/lose count 17.5/1.0. Average score (9.0)
Win/lose count 18.5/2.0. Average score (10.875)
Win/lose count 28.0/2.0. Average score (13.9)
Win/lose count 23.0/4.0. Average score (14.75)
Win/lose count 21.5/1.0. Average score (15.571428571428571)
Win/lose count 21.0/1.0. Average score (16.125)
Win/lose count 23.5/1.0. Average score (16.833333333333332)
Win/lose count 28.0/0. Average score (17.95)
Win/lose count 26.0/5.0. Average score (18.227272727272727)
Final score: 20.05


We do see an improvement in the performance but it is still not a gamewinner as it eats the poison at times and leaves a cheese or two